In [1]:
# installs
# !pip install --quiet sentence-splitter regex
# !pip install --quiet gensim

In [2]:
# imports
import wget
import os
from sentence_splitter import SentenceSplitter
import regex
import gensim

In [3]:
### functions ###

# load file
def download_file(url, filename):
    if not os.path.exists(filename):
        wget.download(url, filename)
    else:
        print(f"{filename} already exists. Skipping download.")

In [4]:
download_file('http://dl.turkunlp.org/TKO_7095_2023/fiwiki-20221120-sample.txt', 'fiwiki-sample.txt')

fiwiki-sample.txt already exists. Skipping download.


In [5]:
### split into sentences

# first into paragraphs
with open('fiwiki-sample.txt', 'r', encoding='utf-8') as file:
    paragraphs = file.readlines()

# initialize splitter and split first 10 000 paragraphs into sentences
splitter = SentenceSplitter(language='fi') 
sentences = [s for p in paragraphs[:100000] for s in splitter.split(p)]

In [6]:
### split into tokens

# sequence of alphanumeric characters followed by a space
TOKENIZE_RE = regex.compile(r'([[:alnum:]]+|\S)')

# tokenize sentences
tokenized = [TOKENIZE_RE.findall(s) for s in sentences]

# print first 10 tokenized sentences
for t in tokenized[:10]:
    print(t)

['Patrick', 'Joseph', 'Leahy', '(', 's', '.']
['31', '.', 'maaliskuuta', '1940', 'Montpelier', ',', 'Vermont', ')', 'on', 'yhdysvaltalainen', 'demokraattisen', 'puolueen', 'poliitikko', '.']
['Leahy', 'toimii', 'Yhdysvaltain', 'senaatin', 'president', 'pro', 'temporena', 'eli', 'de', 'facto', 'senaatin', 'varapresidenttinä', '.']
['Hän', 'on', 'toiminut', 'Vermontin', 'osavaltion', 'senaattorina', 'vuodesta', '1975', '.']
['Grassley', 'myös', 'toimi', 'senaatin', 'president', 'pro', 'temporena', 'joulukuusta', '2012', 'tammikuuhun', '2015', '.']
['Hän', 'on', 'ollut', 'myös', 'senaatin', 'oikeusvaliokunnan', 'puheenjohtaja', '.']
['Elävä', 'kuollut', 'eli', 'epäkuollut', 'tarkoittaa', 'yleisesti', 'erilaisia', 'taruolentoja', ',', 'jotka', 'ovat', 'heränneet', 'kuolleista', 'takaisin', 'elävien', 'maailmaan', '.']
['Populaarikulttuurissa', 'tunnetuimpia', 'eläviä', 'kuolleita', 'ovat', 'vampyyrit', 'ja', 'zombit', '.']
['Sanan', "'", 'epäkuollut', "'", 'kehitti', 'kääntäjä', 'Kersti', 

We notice that the sentences are split quite poorly, but that's okay.

In [7]:
# build gensim word2vec model that creates embeddings for words

model = gensim.models.Word2Vec(
    tokenized,
    vector_size=100,
    window=5,
    min_count=1,
    workers=10)

In [8]:
# find most similar words

words = ['hyvä', 'huono', 'koira', 'kissa', 'kuningas', 'kuningatar']

print("Most similar words:\n")
for word in words:
    print(f"{word}:\n\n{[x[0] for x in model.wv.most_similar(word, topn=10)]}\n\n ------- \n")

Most similar words:

hyvä:

['vahva', 'vaikeaa', 'todella', 'selkeä', 'huono', 'voimakas', 'vaikea', 'todellinen', 'älykäs', 'epäselvää']

 ------- 

huono:

['selvä', 'heikko', 'hankala', 'vahva', 'selkeä', 'tarkka', 'nopea', 'epävarma', 'riippuvainen', 'tehokas']

 ------- 

koira:

['sairaus', 'ihmiselle', 'paha', 'tauti', 'prosessi', 'totuus', 'helppo', 'ominaisuus', 'laite', 'teoria']

 ------- 

kissa:

['näköinen', 'tekstissä', 'puuhun', 'hevonen', 'menneisyyden', 'kuvaan', 'abstrakti', 'paha', 'voimana', 'tuoksu']

 ------- 

kuningas:

['keisari', 'Kaarle', 'Aleksanteri', 'kuninkaan', 'Ludvig', 'kuningatar', 'paavi', 'kuninkaaksi', 'Paavali', 'III']

 ------- 

kuningatar:

['prinssi', 'Juhana', 'Eerik', 'Aleksanteri', 'Katariina', 'Vilhelm', 'Elisabet', 'Fredrik', 'Nikolai', 'Annan']

 ------- 

